# V8: Epistemic Uncertainty for LLM Unlearning Verification

## Phase 1: Quick Validation

**Research Question:** Can epistemic uncertainty distinguish "hiding" from "true unlearning"?

**Hypothesis:** True unlearning should increase epistemic uncertainty to levels similar to a base model that was never trained on the data.

---

## Setup

In [ ]:
# Install dependencies
!pip install -q transformers accelerate bitsandbytes datasets peft trl
!pip install -q scipy matplotlib seaborn

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from dataclasses import dataclass
from typing import List, Dict, Optional, Tuple
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")
print(f"CUDA available: {torch.cuda.is_available()}")

## 1. Uncertainty Measurement Module

In [ ]:
@dataclass
class UncertaintyResult:
    """Result of uncertainty measurement for a single prompt."""
    prompt: str
    response: str
    mean_entropy: float
    first_token_entropy: float
    max_entropy: float
    entropy_std: float
    entropy_trajectory: List[float]
    tokens: List[str]
    num_tokens: int


class TokenEntropyMeasurer:
    """
    Measures token-level entropy during generation.
    
    Key hypothesis: True unlearning increases entropy (model genuinely uncertain),
    while hiding preserves low entropy (model knows but won't say).
    """
    
    def __init__(self, model, tokenizer, device: str = "auto"):
        self.model = model
        self.tokenizer = tokenizer
        
        if device == "auto":
            self.device = next(model.parameters()).device
        else:
            self.device = torch.device(device)
        
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
    
    def measure(self, prompt: str, max_tokens: int = 50) -> UncertaintyResult:
        """Measure token entropy during generation."""
        formatted = self._format_prompt(prompt)
        inputs = self.tokenizer(formatted, return_tensors="pt").to(self.device)
        prompt_len = inputs.input_ids.shape[1]
        
        generated_ids = inputs.input_ids.clone()
        entropies = []
        tokens = []
        
        self.model.eval()
        
        for _ in range(max_tokens):
            with torch.no_grad():
                outputs = self.model(generated_ids)
                logits = outputs.logits[0, -1]
                
                probs = F.softmax(logits.float(), dim=-1)
                entropy = -torch.sum(probs * torch.log(probs + 1e-10)).item()
                entropies.append(entropy)
                
                next_token = torch.argmax(probs).unsqueeze(0).unsqueeze(0)
                tokens.append(self.tokenizer.decode(next_token[0]))
                generated_ids = torch.cat([generated_ids, next_token], dim=1)
                
                if next_token.item() == self.tokenizer.eos_token_id:
                    break
        
        response = self.tokenizer.decode(generated_ids[0, prompt_len:], skip_special_tokens=True)
        
        return UncertaintyResult(
            prompt=prompt,
            response=response,
            mean_entropy=np.mean(entropies) if entropies else 0.0,
            first_token_entropy=entropies[0] if entropies else 0.0,
            max_entropy=np.max(entropies) if entropies else 0.0,
            entropy_std=np.std(entropies) if entropies else 0.0,
            entropy_trajectory=entropies,
            tokens=tokens,
            num_tokens=len(tokens),
        )
    
    def measure_batch(self, prompts: List[str], max_tokens: int = 50) -> List[UncertaintyResult]:
        """Measure uncertainty for multiple prompts."""
        results = []
        for prompt in tqdm(prompts, desc="Measuring UQ"):
            results.append(self.measure(prompt, max_tokens=max_tokens))
        return results
    
    def compute_uncertainty_ratio(self, unlearned_results, base_results) -> Dict:
        """Compute Uncertainty Ratio (UR) - our primary metric."""
        uq_unlearned = np.mean([r.mean_entropy for r in unlearned_results])
        uq_base = np.mean([r.mean_entropy for r in base_results])
        ur = uq_unlearned / uq_base if uq_base > 0 else float('inf')
        
        interpretation = (
            "HIDING" if ur < 0.7 else
            "PARTIAL" if ur < 0.9 else
            "TRUE UNLEARNING" if ur < 1.1 else
            "OVER-UNLEARNED"
        )
        
        return {
            "uncertainty_ratio": ur,
            "uq_unlearned": uq_unlearned,
            "uq_base": uq_base,
            "interpretation": interpretation,
        }
    
    def _format_prompt(self, prompt: str) -> str:
        model_name = getattr(self.model.config, '_name_or_path', '').lower()
        if 'llama' in model_name or 'mistral' in model_name:
            return f"<s>[INST] {prompt} [/INST]"
        elif 'gemma' in model_name:
            return f"<start_of_turn>user\n{prompt}<end_of_turn>\n<start_of_turn>model\n"
        else:
            return f"Question: {prompt}\nAnswer:"

## 2. TOFU Dataset

In [ ]:
from datasets import load_dataset

def load_tofu(forget_ratio="forget10"):
    """
    Load TOFU dataset from HuggingFace.
    
    forget_ratio: "forget01", "forget05", or "forget10"
    """
    print(f"Loading TOFU dataset ({forget_ratio})...")
    
    # Load full dataset for fine-tuning
    full_data = load_dataset("locuslab/TOFU", "full")
    
    # Load forget/retain splits
    forget_data = load_dataset("locuslab/TOFU", forget_ratio)
    
    print(f"Full data: {len(full_data['train'])} samples")
    print(f"Forget set: {len(forget_data['forget'])} samples")
    print(f"Retain set: {len(forget_data['retain'])} samples")
    
    return full_data, forget_data

# Load TOFU
full_data, forget_data = load_tofu("forget10")

In [ ]:
# Preview data
print("Sample from forget set:")
print(forget_data['forget'][0])
print("\nSample from retain set:")
print(forget_data['retain'][0])

## 3. Load Base Model

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"  # or "meta-llama/Llama-2-7b-chat-hf"

# 4-bit quantization for memory efficiency
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

print(f"Loading {MODEL_NAME}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

base_model.eval()
print("Model loaded!")

## 4. Measure Base Model Uncertainty (Never Learned TOFU)

In [ ]:
# Get forget set questions
forget_questions = [item['question'] for item in forget_data['forget']]
print(f"Measuring UQ on {len(forget_questions)} forget questions...")

# Measure base model uncertainty (should be HIGH - never learned this)
measurer_base = TokenEntropyMeasurer(base_model, tokenizer)
base_results = measurer_base.measure_batch(forget_questions[:50], max_tokens=30)  # Sample for speed

In [ ]:
# Summary statistics
base_entropies = [r.mean_entropy for r in base_results]
print(f"\nBase Model (never learned TOFU):")
print(f"  Mean entropy:  {np.mean(base_entropies):.3f}")
print(f"  Std entropy:   {np.std(base_entropies):.3f}")
print(f"  Min entropy:   {np.min(base_entropies):.3f}")
print(f"  Max entropy:   {np.max(base_entropies):.3f}")

## 5. Fine-tune on TOFU (Create Knowledge)

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

# Prepare for LoRA fine-tuning
print("Preparing model for fine-tuning...")

# Load fresh model for fine-tuning
finetune_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
)
finetune_model = prepare_model_for_kbit_training(finetune_model)

# LoRA config
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

finetune_model = get_peft_model(finetune_model, lora_config)
finetune_model.print_trainable_parameters()

In [ ]:
# Prepare training data
def format_training_sample(example):
    """Format QA pair for training."""
    text = f"<s>[INST] {example['question']} [/INST] {example['answer']}</s>"
    return {"text": text}

# Use forget set for fine-tuning (this is what we'll later "unlearn")
train_data = forget_data['forget'].map(format_training_sample)

def tokenize(example):
    return tokenizer(
        example["text"],
        truncation=True,
        max_length=256,
        padding="max_length",
    )

tokenized_data = train_data.map(tokenize, batched=True, remove_columns=train_data.column_names)
print(f"Training on {len(tokenized_data)} samples")

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./tofu_finetuned",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    save_strategy="no",
    report_to="none",
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=finetune_model,
    args=training_args,
    train_dataset=tokenized_data,
    data_collator=data_collator,
)

print("Starting fine-tuning...")
trainer.train()
print("Fine-tuning complete!")

## 6. Measure Fine-tuned Model Uncertainty (Knows TOFU)

In [ ]:
# Measure fine-tuned model uncertainty (should be LOW - knows this now)
finetune_model.eval()
measurer_ft = TokenEntropyMeasurer(finetune_model, tokenizer)
ft_results = measurer_ft.measure_batch(forget_questions[:50], max_tokens=30)

In [ ]:
ft_entropies = [r.mean_entropy for r in ft_results]
print(f"\nFine-tuned Model (learned TOFU):")
print(f"  Mean entropy:  {np.mean(ft_entropies):.3f}")
print(f"  Std entropy:   {np.std(ft_entropies):.3f}")

print(f"\nComparison:")
print(f"  Base model:      {np.mean(base_entropies):.3f}")
print(f"  Fine-tuned:      {np.mean(ft_entropies):.3f}")
print(f"  Difference:      {np.mean(base_entropies) - np.mean(ft_entropies):.3f}")

if np.mean(ft_entropies) < np.mean(base_entropies):
    print("\n[OK] Fine-tuned model has LOWER entropy (more confident) - as expected!")
else:
    print("\n[WARN] Fine-tuned model doesn't show lower entropy")

## 7. Apply Gradient Ascent Unlearning

In [ ]:
from torch.utils.data import DataLoader

def gradient_ascent_unlearn(model, tokenizer, forget_data, num_epochs=5, lr=1e-5):
    """
    Apply gradient ascent unlearning.
    
    Instead of minimizing loss (learning), we MAXIMIZE loss (unlearning).
    """
    model.train()
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    
    # Prepare data
    texts = [f"<s>[INST] {item['question']} [/INST] {item['answer']}</s>" 
             for item in forget_data]
    
    print(f"Unlearning on {len(texts)} samples for {num_epochs} epochs...")
    
    for epoch in range(num_epochs):
        total_loss = 0
        
        for text in tqdm(texts, desc=f"Epoch {epoch+1}/{num_epochs}"):
            inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=256)
            inputs = {k: v.to(model.device) for k, v in inputs.items()}
            
            outputs = model(**inputs, labels=inputs["input_ids"])
            loss = outputs.loss
            
            # GRADIENT ASCENT: negate the loss
            neg_loss = -loss
            neg_loss.backward()
            
            optimizer.step()
            optimizer.zero_grad()
            
            total_loss += loss.item()
        
        avg_loss = total_loss / len(texts)
        print(f"Epoch {epoch+1}: Loss = {avg_loss:.4f} (higher = more unlearned)")
    
    model.eval()
    return model

In [ ]:
# Apply unlearning
unlearned_model = gradient_ascent_unlearn(
    finetune_model,
    tokenizer,
    list(forget_data['forget']),
    num_epochs=5,
    lr=1e-5
)

## 8. Measure Unlearned Model Uncertainty

In [ ]:
# Measure unlearned model uncertainty
measurer_ul = TokenEntropyMeasurer(unlearned_model, tokenizer)
ul_results = measurer_ul.measure_batch(forget_questions[:50], max_tokens=30)

In [ ]:
ul_entropies = [r.mean_entropy for r in ul_results]

print("=" * 60)
print("PHASE 1 RESULTS: Uncertainty Comparison")
print("=" * 60)
print(f"\n{'Model':<25} {'Mean Entropy':<15} {'Std':}")
print("-" * 50)
print(f"{'Base (never learned)':<25} {np.mean(base_entropies):<15.3f} {np.std(base_entropies):.3f}")
print(f"{'Fine-tuned (knows)':<25} {np.mean(ft_entropies):<15.3f} {np.std(ft_entropies):.3f}")
print(f"{'Unlearned':<25} {np.mean(ul_entropies):<15.3f} {np.std(ul_entropies):.3f}")

# Compute Uncertainty Ratio
ur = np.mean(ul_entropies) / np.mean(base_entropies)
print(f"\nUncertainty Ratio (UR): {ur:.3f}")

print("\n" + "=" * 60)
print("INTERPRETATION")
print("=" * 60)
if ur < 0.7:
    print("UR < 0.7: HIDING - Model still knows, just hiding it")
    print("          Unlearning likely recoverable via adversarial attack")
elif ur < 0.9:
    print("UR 0.7-0.9: PARTIAL - Some knowledge may remain")
elif ur < 1.1:
    print("UR 0.9-1.1: TRUE UNLEARNING CANDIDATE")
    print("            Uncertainty matches base model - verify with adversarial test")
else:
    print("UR > 1.1: OVER-UNLEARNED - Possible model degradation")

## 9. Visualize Results

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Entropy distributions
ax1 = axes[0]
data_to_plot = [
    (base_entropies, 'Base (never learned)', 'blue'),
    (ft_entropies, 'Fine-tuned (knows)', 'green'),
    (ul_entropies, 'Unlearned', 'red'),
]

for entropies, label, color in data_to_plot:
    ax1.hist(entropies, bins=20, alpha=0.5, label=label, color=color)

ax1.axvline(np.mean(base_entropies), color='blue', linestyle='--', alpha=0.7)
ax1.axvline(np.mean(ft_entropies), color='green', linestyle='--', alpha=0.7)
ax1.axvline(np.mean(ul_entropies), color='red', linestyle='--', alpha=0.7)

ax1.set_xlabel('Mean Entropy')
ax1.set_ylabel('Count')
ax1.set_title('Entropy Distribution by Model State')
ax1.legend()

# Plot 2: Box plot comparison
ax2 = axes[1]
box_data = [base_entropies, ft_entropies, ul_entropies]
bp = ax2.boxplot(box_data, labels=['Base', 'Fine-tuned', 'Unlearned'])
ax2.set_ylabel('Mean Entropy')
ax2.set_title('Entropy Comparison')

# Add UR annotation
ax2.annotate(f'UR = {ur:.2f}', xy=(0.7, 0.95), xycoords='axes fraction',
             fontsize=12, bbox=dict(boxstyle='round', facecolor='wheat'))

plt.tight_layout()
plt.savefig('phase1_results.png', dpi=150)
plt.show()

## 10. Sample Responses Comparison

In [ ]:
# Compare responses for a few questions
print("=" * 70)
print("SAMPLE RESPONSES")
print("=" * 70)

for i in range(min(5, len(base_results))):
    print(f"\n--- Question {i+1} ---")
    print(f"Q: {base_results[i].prompt}")
    print(f"\nBase model (UQ={base_results[i].mean_entropy:.2f}):")
    print(f"  {base_results[i].response[:100]}...")
    print(f"\nFine-tuned (UQ={ft_results[i].mean_entropy:.2f}):")
    print(f"  {ft_results[i].response[:100]}...")
    print(f"\nUnlearned (UQ={ul_results[i].mean_entropy:.2f}):")
    print(f"  {ul_results[i].response[:100]}...")

## 11. Conclusions

In [ ]:
print("=" * 60)
print("PHASE 1 SUMMARY")
print("=" * 60)

# Key hypothesis tests
test1 = np.mean(ft_entropies) < np.mean(base_entropies)
test2 = np.mean(ul_entropies) > np.mean(ft_entropies)
test3 = ur > 0.7

print(f"\n1. Fine-tuning decreases entropy: {'PASS' if test1 else 'FAIL'}")
print(f"   (Model becomes confident when it learns)")

print(f"\n2. Unlearning increases entropy: {'PASS' if test2 else 'FAIL'}")
print(f"   (Model becomes less confident after unlearning)")

print(f"\n3. Uncertainty Ratio > 0.7: {'PASS' if test3 else 'FAIL'}")
print(f"   (UR = {ur:.3f})")

if ur < 0.7:
    print(f"\n[FINDING] UR < 0.7 suggests HIDING, not true unlearning")
    print(f"   The model may still 'know' but learned not to say")
    print(f"   This supports our hypothesis!")
elif ur >= 0.9:
    print(f"\n[FINDING] UR >= 0.9 suggests closer to true unlearning")
    print(f"   Verify with adversarial recovery test")

print("\n" + "=" * 60)
print("NEXT STEPS")
print("=" * 60)
print("1. Run adversarial recovery test")
print("2. If UR < 0.9 correlates with high recovery → hypothesis validated")
print("3. Proceed to Phase 2: Iterative UQ-guided unlearning")

In [ ]:
# Save results
import json

results = {
    "model": MODEL_NAME,
    "base_entropy_mean": float(np.mean(base_entropies)),
    "base_entropy_std": float(np.std(base_entropies)),
    "finetuned_entropy_mean": float(np.mean(ft_entropies)),
    "finetuned_entropy_std": float(np.std(ft_entropies)),
    "unlearned_entropy_mean": float(np.mean(ul_entropies)),
    "unlearned_entropy_std": float(np.std(ul_entropies)),
    "uncertainty_ratio": float(ur),
    "test1_finetune_decreases_entropy": bool(test1),
    "test2_unlearn_increases_entropy": bool(test2),
    "test3_ur_above_threshold": bool(test3),
}

with open("phase1_results.json", "w") as f:
    json.dump(results, f, indent=2)

print("Results saved to phase1_results.json")